## This notebook randomly generates student performance over time and visualizes/analyzes the data.
The goal is to create a tool that visualizes the data of students so teachers can understand their progress over time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def student_performace(start,end,deviation,days):
    time = np.arange(days) #create date range
    av_over_time = np.linspace(start,end,days) #creates linear average change. A custom function
                                                #can be created to change the average to follow
    grade_arr = []                             #some other function
    day_arr = []
    
    for day in time:
        if np.random.random()<0.3: #randomly generate assessments
            day_arr.append(day) #add day to list
            grade_arr.append(np.random.normal(av_over_time[day],deviation)) #generate assessment score
                                                                           #with deviation
    return day_arr,grade_arr

In [3]:
time,performance = student_performace(45,60,2,210)

### Next we will calculate the rolling average

In [4]:
def rolling_average(y_vals):
    rolling_average = []
    yval_len = len(y_vals)
    
    rolling_average.append(y_vals[0]) #start with first assessment
    
    
    for index in np.arange(1,6): #calculate rolling average for first few assessments
        vals = y_vals[0:index]
        rolling_average.append(sum(vals)/len(vals))
    
    for index in np.arange(6,yval_len): #calculate rolling average for the rest of the assessments
        vals = y_vals[index-6:index]
        rolling_average.append(sum(vals)/len(vals))
    
    
    return rolling_average

In [5]:
rolling_average = rolling_average(performance)

In [6]:
#I can be picky about my color palletes
node_colors = ["#a50026","#d73027","#f46d43","#fdae61","#fee08b","#ffffbf","#d9ef8b","#a6d96a","#66bd63","#1a9850","#006837"]

### We will calculate the difference between the rolling average and the assessments. This will allow us to grade the student assessments relative to their recent performance.
We will grade the assessment from 0-10 based on their distance from the rolling average.

In [7]:
difference = np.array(performance)-np.array(rolling_average)

In [8]:
a_min = 5/min(difference)
a_max = 5/max(difference)

In [9]:
for index in range(len(difference)):
    if difference[index]>0:
        difference[index] = int(a_max*difference[index])
    if difference[index]<0:
        difference[index] = -int(a_min*difference[index])

difference = difference+abs(min(difference))

In [10]:
difference = difference.astype("int32")

In [11]:
colors = [] #assign colors to scores
for dif in difference:
    colors.append(node_colors[dif])

In [12]:
#import bokeh packages for scoring
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.plotting import figure, output_file, show

Loading BokehJS ...

In [13]:
# output to static HTML file
output_file("line.html")


data = {'time' : time,
        'score':performance,
       'rolling average':rolling_average,
       'color':colors,
       'difference':difference}


p = figure(width=800, height=400,
          toolbar_location=None, tools="hover", tooltips="Adjusted Score: @difference",
          title="Student Performance Over Time",x_axis_label='Day',
          y_axis_label='Assessment Score')


p.title.text_font_size = '18pt'

p.line("time", "score", line_width=2,color = "black",source=data,alpha=0.4)


p.line("time", "rolling average", line_width=4,color = "firebrick",alpha = 0.8,line_dash=[6, 3],source=data)

# add a circle renderer with a size, color, and alpha
p.circle("time","score", size=15, color="color",line_color = "black",source=data)

# show the results
show(p)

<img src="gradeovertime.png" width="800" />

Next we have a simple bar plot for comparing the average of a student vs the rest of the class, compared over assessment type.

In [14]:
# output to static HTML file
output_file("bar_plot.html")

categories = ["Labs","Assignments","Quizzes","Tests","Exam"]
data = {'Assessment Category' : categories,
        'Class Average':[85,79,80,74,78],
       'Student Average':[94,88,95,90,85]}

source = ColumnDataSource(data=data)

p = figure(x_range = categories, y_range = (0, 120), width=800, height=400,toolbar_location=None,
          title="Student and Class Averages",x_axis_label='Assessment Type',
          y_axis_label='Assessment Score')

p.title.text_font_size = '18pt'


p.vbar(x=dodge('Assessment Category', -0.125, range=p.x_range), top='Class Average', width=0.2,source=source,
       color="firebrick", legend_label="Class Average",line_color="black")

p.vbar(x=dodge('Assessment Category',  0.125,  range=p.x_range), top='Student Average', width=0.2, source=source,
       color="Skyblue", legend_label="Student Average",line_color="black")

p.toolbar.active_drag = None
p.toolbar.active_scroll = None
p.toolbar.active_tap = None

# show the results
show(p)

<img src="barplots.png" width="800" />